# Group-4

## Problem Statement

---

There is a new innovation in the financial industry called Neo-Banks: new aged digital-only banks without physical branches. We thought that there should be some sort of intersection between these new-age banks, cryptocurrency, and the data world. So we decide to launch a new initiative, Data Bank!

> **Data Bank runs just like any other digital bank, but it isn’t only for banking activities, they also have the world’s most secure distributed data storage platform!** Customers are allocated cloud data storage limits which are directly linked to how much money they have in their accounts. There are a few interesting caveats that go with this business model, and this is where the Data Bank team needs your help!
> 

The management team at Data Bank wants to increase its total customer base but also needs some help tracking just how much data storage its customers will need. This case study is all about calculating metrics, and growth and helping the business analyze their data in a smart way to better forecast and plan for their future developments!

## Avaliable Data

---

Our team has prepared a data model for this case study to get you familiar with their data.

![Untitled](https://flannel-xylophone-a44.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F65959e22-f524-45f7-b911-f85842b6ca73%2FUntitled.png?table=block&id=418f199c-761f-4443-9798-abf313bf18ba&spaceId=83b5e96f-5c59-4a23-b33b-0fafc3cb289d&width=1340&userId=&cache=v2)

### **Table 1: Regions**

Just like popular cryptocurrency platforms, Data Bank is also run off a network of nodes where both money and data is stored across the globe. In a traditional banking sense, you can think of these nodes as bank branches or stores that exist around the world. 

### **Table 2: Customer Nodes**

Customers are randomly distributed across the nodes according to their region, this also specifies exactly which node contains both their cash and data. This random distribution changes frequently to reduce the risk of hackers getting into Data Bank’s system and stealing customer’s money and data!

### **Table 3: Customer Transactions**

This table stores all customer deposits, withdrawals and purchases made using their Data Bank debit card.

**Link to download data:** 

https://drive.google.com/drive/folders/1VNTTdExilLfkxkyZrWnWYXdkdSKt26kr?usp=share_link

## Answers

---

### Create MySQL Connectors & `show` Function

In [3]:
import mysql.connector
import pandas as pd

def show(db, qry):
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Einstein0401$',
        database=db
    )

    executor = connection.cursor()
    executor.execute(qry)
    table = executor.fetchall()

    connection.close()
    return pd.DataFrame(data=table, columns=executor.column_names)

### Question 1: **Customer Nodes Exploration**

1. How many unique nodes are there on the Data Bank system?

In [4]:
query = '''
SELECT  customer_nodes.region_id, 
        regions.region_name,
        count(distinct customer_nodes.node_id) AS unique_nodes
FROM customer_nodes JOIN regions ON (customer_nodes.region_id = regions.region_id)
GROUP BY customer_nodes.region_id, regions.region_name
ORDER BY customer_nodes.region_id;
'''
show('data_bank', query)

,region_id,region_name,unique_nodes
0,1,Australia,5
1,2,America,5
2,3,Africa,5
3,4,Asia,5
4,5,Europe,5


2. What is the number of nodes per region?

In [5]:
query = '''
SELECT	regions.region_id,
		regions.region_name,
		COUNT(node_id) AS number_of_nodes
FROM customer_nodes, regions
WHERE customer_nodes.region_id = regions.region_id
GROUP BY region_id, region_name;
'''
show('data_bank', query)

,region_id,region_name,number_of_nodes
0,1,Australia,770
1,2,America,735
2,3,Africa,714
3,4,Asia,665
4,5,Europe,616


3. How many customers are allocated to each region?

In [6]:
query = '''
SELECT	regions.region_id,
		region_name,
		COUNT(DISTINCT customer_id) AS customers_allocated
FROM customer_nodes, regions
WHERE customer_nodes.region_id = regions.region_id
GROUP BY region_id, region_name
ORDER BY region_id;
'''
show('data_bank', query)

,region_id,region_name,customers_allocated
0,1,Australia,110
1,2,America,105
2,3,Africa,102
3,4,Asia,95
4,5,Europe,88


4. How many days on average are customers reallocated to a different node?

In [7]:
query = '''
SELECT ROUND(AVG(DATEDIFF(end_date, start_date))) AS average_days
FROM customer_nodes
WHERE YEAR(end_date) <> 9999;
'''
show('data_bank', query)

,average_days
0,15


5. What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [8]:
query = ''' 
WITH cte AS (
SELECT	customer_id,
		regions.region_id AS region_id,
        regions.region_name AS region_name,
		AVG(DATEDIFF(end_date, start_date)) AS average_days,
        NTILE(100) OVER (ORDER BY AVG(DATEDIFF(end_date, start_date))) AS average_days_percentile
FROM customer_nodes, regions
WHERE YEAR(end_date) <> 9999
GROUP BY customer_id, regions.region_id, regions.region_name
)
SELECT DISTINCT
	region_id,
	region_name,
    ROUND(((SELECT MAX(average_days) FROM cte WHERE average_days_percentile = 50) 
		+ (SELECT MIN(average_days) FROM cte WHERE average_days_percentile = 51))
		/ 2) AS average_days_median,
    ROUND((SELECT DISTINCT average_days FROM cte WHERE average_days_percentile = 80)) AS average_days_80th_percentile
FROM cte;
'''
show('data_bank', query)

,region_id,region_name,average_days_median,average_days_80th_percentile
0,5,Europe,15,18
1,4,Asia,15,18
2,3,Africa,15,18
3,2,America,15,18
4,1,Australia,15,18


In [9]:
query = ''' 
WITH cte AS (
SELECT	customer_id,
		regions.region_id AS region_id,
        regions.region_name AS region_name,
		AVG(DATEDIFF(end_date, start_date)) AS average_days,
        NTILE(100) OVER (ORDER BY AVG(DATEDIFF(end_date, start_date))) AS average_days_percentile
FROM customer_nodes, regions
WHERE YEAR(end_date) <> 9999
GROUP BY customer_id, regions.region_id, regions.region_name
)
SELECT DISTINCT
	region_id,
	region_name,
	ROUND(average_days) AS average_days,
	average_days_percentile
FROM cte
WHERE average_days_percentile = 95;
'''
show('data_bank', query)

,region_id,region_name,average_days,average_days_percentile
0,5,Europe,20,95
1,4,Asia,20,95
2,3,Africa,20,95
3,2,America,20,95
4,1,Australia,20,95
5,5,Europe,21,95
6,4,Asia,21,95
7,3,Africa,21,95
8,2,America,21,95
9,1,Australia,21,95


### Question 2: **Customer Transactions**

1. What is the unique count and total amount for each transaction type?

In [10]:
query = ''' 
SELECT DISTINCT
	txn_type,
	COUNT(txn_type) AS unique_count,
	SUM(txn_amount) AS total_amount
		FROM customer_transactions
        GROUP BY txn_type
        ORDER BY unique_count DESC;
'''
show('data_bank', query)

,txn_type,unique_count,total_amount
0,deposit,2671,1359168
1,purchase,1617,806537
2,withdrawal,1580,793003


2. What is the average total historical deposit counts and amounts for all customers?

In [11]:
query = ''' 
WITH cte AS (
SELECT	customer_id,
	COUNT(txn_amount) AS unique_count,
        SUM(txn_amount) AS total_amount
FROM customer_transactions
WHERE txn_type = 'deposit'
GROUP BY customer_id
)
SELECT	ROUND(AVG(unique_count)) AS average_deposit_count,
		ROUND(AVG(total_amount)) AS average_total_deposit
FROM cte;
'''
show('data_bank', query)

,average_deposit_count,average_total_deposit
0,5,2718


3. For each month, how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

In [12]:
query = ''' 
WITH cte AS (
SELECT 	table1.customer_id,
		table1.txn_month AS month_id, 
        table1.month_name AS name,
        table1.txn_type AS type,
        table1.deposit_count,
        table2.txn_month,
        table2.month_name,
        table2.txn_type,
        table2.count
FROM (SELECT	customer_id,
				MONTH(txn_date) AS txn_month,
				MONTHNAME(txn_date) AS month_name,
				txn_type,
				COUNT(txn_type) AS deposit_count
		FROM customer_transactions
		WHERE txn_type = 'deposit'
		GROUP BY customer_id, txn_month, month_name, txn_type
		HAVING deposit_count > 1)
			AS table1,
	(SELECT	customer_id,
			MONTH(txn_date) AS txn_month,
			MONTHNAME(txn_date) AS month_name,
			txn_type,
			COUNT(txn_type) AS count
		FROM customer_transactions
		WHERE txn_type = 'purchase' OR txn_type = 'withdrawal'
		GROUP BY customer_id, txn_month, month_name, txn_type
		HAVING count = 1)
			AS table2
WHERE table1.customer_id = table2.customer_id
)
SELECT 	month_id,
		name AS month_name,
        COUNT(DISTINCT customer_id) AS customer_count
FROM cte
GROUP BY month_id, name;
'''
show('data_bank', query)

,month_id,month_name,customer_count
0,1,January,214
1,2,February,192
2,3,March,200
3,4,April,82


4. What is the closing balance for each customer at the end of the month?

In [13]:
query = ''' 
WITH cte1 AS (
SELECT	customer_id AS id1,
		MONTH(txn_date) AS txn_month1,
        MONTHNAME(txn_date) AS month_name1,
		SUM(txn_amount) AS total_deposit
FROM customer_transactions
WHERE txn_type = 'deposit'
GROUP BY customer_id, txn_month1, month_name1
),
cte2 AS (
SELECT	customer_id AS id2,
		MONTH(txn_date) AS txn_month2,
		MONTHNAME(txn_date) AS month_name2,
		SUM(txn_amount) AS total_withdrawal
FROM customer_transactions
WHERE txn_type = 'withdrawal'
GROUP BY customer_id, txn_month2, month_name2
),
cte3 AS (
SELECT	customer_id AS id3,
		MONTH(txn_date) AS txn_month3,
		MONTHNAME(txn_date) AS month_name3,
		SUM(txn_amount) AS total_purchase
FROM customer_transactions
WHERE txn_type = 'purchase'
GROUP BY customer_id, txn_month3, month_name3
),
cte4 AS (
SELECT	id1 AS customer_id,
		txn_month1 AS txn_month,
		month_name1 AS month_name,
		total_deposit,
        total_withdrawal,
        total_purchase,
		total_deposit - IFNULL(total_withdrawal, 0) - IFNULL(total_purchase, 0) AS balance,
        SUM(total_deposit - IFNULL(total_withdrawal, 0) - IFNULL(total_purchase, 0)) OVER(PARTITION BY id1) AS closing_balance,
		ROW_NUMBER() OVER (PARTITION BY id1 ORDER BY txn_month1 DESC) AS row_no
FROM cte1
LEFT JOIN cte2 ON id1 = id2 AND txn_month1 = txn_month2
LEFT JOIN cte3 ON id1 = id3 AND txn_month1 = txn_month3
ORDER BY id1
)
SELECT	customer_id,
		txn_month,
		month_name,
        closing_balance
FROM cte4
WHERE row_no = 1;
'''
show('data_bank', query)

,customer_id,txn_month,month_name,closing_balance
0,1,3,March,-640
1,2,3,March,610
2,3,4,April,637
3,4,1,January,848
4,5,3,March,-1923
...,...,...,...,...
495,496,3,March,697
496,497,4,April,2680
497,498,4,April,3488
498,499,3,March,599


5. What is the percentage of customers who increase their closing balance by more than 5%?

In [14]:
query = ''' 
WITH cte1 AS (
SELECT	customer_id AS id1,
		MONTH(txn_date) AS txn_month1,
        MONTHNAME(txn_date) AS month_name1,
		SUM(txn_amount) AS total_deposit
FROM customer_transactions
WHERE txn_type = 'deposit'
GROUP BY customer_id, txn_month1, month_name1
),
cte2 AS (
SELECT	customer_id AS id2,
		MONTH(txn_date) AS txn_month2,
		MONTHNAME(txn_date) AS month_name2,
		SUM(txn_amount) AS total_withdrawal
FROM customer_transactions
WHERE txn_type = 'withdrawal'
GROUP BY customer_id, txn_month2, month_name2
),
cte3 AS (
SELECT	customer_id AS id3,
		MONTH(txn_date) AS txn_month3,
		MONTHNAME(txn_date) AS month_name3,
		SUM(txn_amount) AS total_purchase
FROM customer_transactions
WHERE txn_type = 'purchase'
GROUP BY customer_id, txn_month3, month_name3
),
cte4 AS (
SELECT	id1 AS id4,
		txn_month1 AS txn_month4,
		month_name1 AS month_name4,
		total_deposit,
        total_withdrawal,
        total_purchase,
		total_deposit - IFNULL(total_withdrawal, 0) - IFNULL(total_purchase, 0) AS balance,
        SUM(total_deposit - IFNULL(total_withdrawal, 0) - IFNULL(total_purchase, 0)) OVER(PARTITION BY id1) AS closing_balance,
		ROW_NUMBER() OVER (PARTITION BY id1 ORDER BY txn_month1 DESC) AS row_no
FROM cte1
LEFT JOIN cte2 ON id1 = id2 AND txn_month1 = txn_month2
LEFT JOIN cte3 ON id1 = id3 AND txn_month1 = txn_month3
ORDER BY id1
),
cte5 AS (
SELECT	id4 AS id5,
		txn_month4 AS txn_month5,
		month_name4 AS month_name5,
        closing_balance
FROM cte4
WHERE row_no = 1
),
cte6 AS(
SELECT	id4 AS id6,
		txn_month4,
		month_name4,
        balance AS previous_month_balance,
        cte5.closing_balance AS closing_balance,
        cte5.closing_balance / balance AS balance_ratio
FROM cte4
FULL JOIN cte5 ON id4 = id5
WHERE row_no = 1
)
SELECT	(SELECT COUNT(id6) AS customer_count FROM cte6 WHERE balance_ratio > 0.05) AS customer_count_filtered,
		COUNT(id6) AS customer_count,
		ROUND((SELECT COUNT(id6) AS customer_count FROM cte6 WHERE balance_ratio > 0.05) / COUNT(id6) * 100, 1) AS filtered_customer_percentage
FROM cte6;
'''
show('data_bank', query)

,customer_count_filtered,customer_count,filtered_customer_percentage
0,376,500,75.2
